# Class corresponding to rayleigh_block_fading

In [1]:
"""Class for simulating Rayleigh block fading"""

import tensorflow as tf

from sionna.channel import ChannelModel

from sionna.channel import OFDMChannel

from sionna.ofdm import ResourceGrid 

2024-04-24 00:38:44.969866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 00:38:45.853569: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## official explain

In [2]:
"""RayleighBlockFading(num_rx, num_rx_ant, num_tx, num_tx_ant, dtype=tf.complex64)

    Generate channel impulse responses corresponding to a Rayleigh block
    fading channel model.

    The channel impulse responses generated are formed of a single path with
    zero delay and a normally distributed fading coefficient.
    All time steps of a batch example share the same channel coefficient
    (block fading).

    This class can be used in conjunction with the classes that simulate the
    channel response in time or frequency domain, i.e.,
    :class:`~sionna.channel.OFDMChannel`,
    :class:`~sionna.channel.TimeChannel`,
    :class:`~sionna.channel.GenerateOFDMChannel`,
    :class:`~sionna.channel.ApplyOFDMChannel`,
    :class:`~sionna.channel.GenerateTimeChannel`,
    :class:`~sionna.channel.ApplyTimeChannel`.

    Parameters
    ----------

    num_rx : int
        Number of receivers (:math:`N_R`)

    num_rx_ant : int
        Number of antennas per receiver (:math:`N_{RA}`)

    num_tx : int
        Number of transmitters (:math:`N_T`)

    num_tx_ant : int
        Number of antennas per transmitter (:math:`N_{TA}`)

    dtype : tf.DType
        Complex datatype to use for internal processing and output.
        Defaults to `tf.complex64`.

    Input
    -----
    batch_size : int
        Batch size

    num_time_steps : int
        Number of time steps

    Output
    -------
    a : [batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths = 1, num_time_steps], tf.complex
        Path coefficients

    tau : [batch size, num_rx, num_tx, num_paths = 1], tf.float
        Path delays [s]
    """

'RayleighBlockFading(num_rx, num_rx_ant, num_tx, num_tx_ant, dtype=tf.complex64)\n\n    Generate channel impulse responses corresponding to a Rayleigh block\n    fading channel model.\n\n    The channel impulse responses generated are formed of a single path with\n    zero delay and a normally distributed fading coefficient.\n    All time steps of a batch example share the same channel coefficient\n    (block fading).\n\n    This class can be used in conjunction with the classes that simulate the\n    channel response in time or frequency domain, i.e.,\n    :class:`~sionna.channel.OFDMChannel`,\n    :class:`~sionna.channel.TimeChannel`,\n    :class:`~sionna.channel.GenerateOFDMChannel`,\n    :class:`~sionna.channel.ApplyOFDMChannel`,\n    :class:`~sionna.channel.GenerateTimeChannel`,\n    :class:`~sionna.channel.ApplyTimeChannel`.\n\n    Parameters\n    ----------\n\n    num_rx : int\n        Number of receivers (:math:`N_R`)\n\n    num_rx_ant : int\n        Number of antennas per rece

## RayleighBlockFading的定义和解释
### ChannelModel
`ChannelModel`是被定义的抽象类，它提供了一个用于通道模型的接口，为`RayleighBlockFading`的创建提供了模版。下面是对代码中各部分的详细解释：

1. `class ChannelModel(ABC):`
   这行代码定义了一个名为`ChannelModel`的类，并且指定它是一个抽象类（`ABC`表示抽象基类），意味着它不能直接实例化，而是用来作为其他类的基类或接口。

2. `ChannelModel()`
   这是类的文档字符串(docstring),用于描述类的作用和用法。在这里描述了`ChannelModel`类的抽象特性和接口定义。

3. `@abstractmethod`
   这是一个装饰器，用于标记一个抽象方法，即`__call__`方法。抽象方法需要在子类中被实现，否则会抛出`NotImplementedError`异常。

4. `def __call__(self,  batch_size, num_time_steps, sampling_frequency):`
   这是一个抽象方法，定义了通道模型的调用方式。它接受`batch_size`（批大小）、`num_time_steps`（时间步数）和`sampling_frequency`（采样频率）等参数，并且返回通道路径的系数和延迟。

总体来说，这段代码定义了一个通道模型的抽象接口，任何实现了这个接口的通道模型都需要提供`__call__`方法来生成通道脉冲响应。
### 类描述
生成**瑞利衰落信道模型**对应的**信道脉冲响应**（*CIR*）。

生成的**信道脉冲响应**由一个具有**零延迟**和**正态分布衰落系数的单路径**组成。一个批次示例的所有时间步骤共享相同的信道系数（块衰落）。

该类可与模拟时域或频域中的信道响应的类一起使用，例如:
- :class:`~sionna.channel.OFDMChannel`
- :class:`~sionna.channel.TimeChannel`
- :class:`~sionna.channel.GenerateOFDMChannel`
- :class:`~sionna.channel.ApplyOFDMChannel`
- :class:`~sionna.channel.GenerateTimeChannel`
- :class:`~sionna.channel.ApplyTimeChannel`

In [3]:
class RayleighBlockFading(ChannelModel):
    def __init__(   self,
                    num_rx,
                    num_rx_ant,
                    num_tx,
                    num_tx_ant,
                    dtype=tf.complex64):

        assert dtype.is_complex, "'dtype' must be complex type"
        self._dtype = dtype

        # We don't set these attributes as private so that the user can update
        # them
        self.num_tx = num_tx
        self.num_tx_ant = num_tx_ant
        self.num_rx = num_rx
        self.num_rx_ant = num_rx_ant
    def __call__(self,  batch_size, num_time_steps, sampling_frequency=None):

        # Delays
        # Single path with zero delay
        delays = tf.zeros([ batch_size,
                            self.num_rx,
                            self.num_tx,
                            1], # Single path
                            dtype=self._dtype.real_dtype)

        # Fading coefficients
        # tf.cast转换数据格式
        std = tf.cast(tf.sqrt(0.5), dtype=self._dtype.real_dtype)
        h_real = tf.random.normal(shape=[   batch_size,
                                            self.num_rx,
                                            self.num_rx_ant,
                                            self.num_tx,
                                            self.num_tx_ant,
                                            1, # One path
                                            1], # Same response over the block
                                            stddev=std,#标准差
                                            dtype = self._dtype.real_dtype)
        h_img = tf.random.normal(shape=[    batch_size,
                                            self.num_rx,
                                            self.num_rx_ant,
                                            self.num_tx,
                                            self.num_tx_ant,
                                            1, # One cluster
                                            1], # Same response over the block
                                            stddev=std,
                                            dtype = self._dtype.real_dtype)
        h = tf.complex(h_real, h_img)
        # Tile the response over the block
        h = tf.tile(h, [1, 1, 1, 1, 1, 1, num_time_steps])
        return h, delays

## __call__()
这段代码是一个Python函数，主要用于生成瑞利衰落信道模型对应的信道脉冲响应。

1. `batch_size` 是批处理大小，表示一次处理的样本数量。
2. `num_time_steps` 是时间步数，表示信道模型的时间长度或采样数。
3. `sampling_frequency` 是采样频率，用于生成时间轴上的信道响应。在这里没有直接使用，可能是因为具体的信道模型不需要此参数。

接下来是具体的处理步骤：
- `delays` 是一个零矩阵，表示信道中单路径的延迟。它的形状是 `[batch_size, num_rx, num_tx, 1]`，即批处理大小、接收端数量、发送端数量、单路径。
- `std` 是标准差，用于生成服从正态分布的随机数。在这里，使用 `tf.sqrt(0.5)` 计算标准差。
- `h_real` 和 `h_img` 分别是实部和虚部的随机数，表示信道的实部和虚部响应。它们的形状是 `[batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, 1, 1]`，即批处理大小、接收端数量、接收端天线数量、发送端数量、发送端天线数量、单路径、单簇。
- `h` 是复数形式的信道响应，由实部和虚部组成。使用 `tf.complex(h_real, h_img)` 将实部和虚部合并为复数形式。
- 最后，使用 `tf.tile` 将信道响应复制扩展到指定的时间步数 `num_time_steps`，以模拟信道在时间上的变化。

整体来说，这段代码生成了一个具有指定参数的瑞利衰落信道模型的信道脉冲响应。

`__call__`这个方法是一个特殊的Python魔术方法，在类中被调用时会触发。在这个方法中，`self` 代表类的实例，可以通过类的实例来调用这个方法。对于上面的代码片段，如果要使用这个方法，需要先创建一个类的实例，然后调用这个实例的 `__call__` 方法，并传入相应的参数。

例如，假设你有一个名为 `channel_model` 的类的实例，你可以像这样调用这个方法：

```python
# 创建一个 ChannelModel 的实例
channel_model = ChannelModel(num_rx=2, num_rx_ant=4, num_tx=2, num_tx_ant=2)

# 调用 __call__ 方法并传入参数
h, delays = channel_model(batch_size=10, num_time_steps=100)
```

在这个例子中，我们通过 `channel_model(batch_size=10, num_time_steps=100)` 的方式调用了 `__call__` 方法，并传入了 `batch_size` 和 `num_time_steps` 这两个参数。

## tf方法
### tf.complex用法

In [4]:
import tensorflow as tf

# 随机生成实部和虚部的张量
h_real = tf.random.normal(shape=[1, 2, 2], stddev=0.1)
h_img = tf.random.normal(shape=[1, 2, 2], stddev=0.1)

# 合并实部和虚部为复数形式的信道响应
h = tf.complex(h_real, h_img)
print(f"{h_real}\n")
print(f"{h_img}\n")
print(f"{h}\n")


[[[-0.02547519  0.04303628]
  [-0.04981866  0.08261553]]]

[[[ 0.07565582 -0.04036371]
  [ 0.25853822 -0.10535242]]]

[[[-0.02547519+0.07565582j  0.04303628-0.04036371j]
  [-0.04981866+0.25853822j  0.08261553-0.10535242j]]]



2024-04-24 00:38:47.832692: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19640 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:21:00.0, compute capability: 8.9


### tf.cast 用法

`tf.cast` 是 TensorFlow 中用于数据类型转换的函数。它的作用是将输入张量转换为指定的数据类型。例如，你可以将一个整数张量转换为浮点数张量，或者将一个布尔张量转换为整数张量等。


In [5]:
import tensorflow as tf

# 创建一个整数张量
x = tf.constant([1, 2, 3])

# 将整数张量转换为浮点数张量
y = tf.cast(x, dtype=tf.float32)

print(y)  # 输出: <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)



在上面的例子中，`tf.cast(x, dtype=tf.float32)` 将整数张量 `x` 转换为了浮点数张量 `y`。`tf.cast` 可以处理各种数据类型的转换，但需要注意的是，有些转换可能会导致数据精度的损失。

## tf.random.normal
`tf.random.normal` 是 TensorFlow 中用于生成服从正态分布（高斯分布）的随机张量的函数。它的语法如下：

```python
tf.random.normal(
    shape,
    mean=0.0,
    stddev=1.0,
    dtype=tf.dtypes.float32,
    seed=None,
    name=None
)
```

参数说明：
- `shape`：生成随机张量的形状。
- `mean`：正态分布的均值，默认为0.0。
- `stddev`：正态分布的标准差，默认为1.0。
- `dtype`：生成张量的数据类型，默认为 `tf.float32`。
- `seed`：随机种子，用于生成随机数的伪随机性。
- `name`：操作的名称。

示例用法：

In [6]:
import tensorflow as tf

# 生成一个形状为 (2, 2) 的随机张量，均值为 0，标准差为 1
x = tf.random.normal(shape=(2, 2))

print(x)

tf.Tensor(
[[0.04834978 0.3841536 ]
 [0.06927757 0.5414761 ]], shape=(2, 2), dtype=float32)


这段代码将生成一个形状为 (2, 2) 的随机张量，其中的元素是从均值为 0、标准差为 1 的正态分布中随机抽样得到的。

### tf.tile
`tf.tile` 是 TensorFlow 中用于复制张量的函数。它的语法如下：

```python
tf.tile(
    input,
    multiples,
    name=None
)
```

参数说明：
- `input`：要复制的张量。
- `multiples`：一个整数列表，表示每个维度上复制的次数。
- `name`：操作的名称。

示例用法：

```python
import tensorflow as tf

# 定义一个形状为 (2, 3) 的张量
x = tf.constant([[1, 2, 3], [4, 5, 6]])

# 在第一个维度上复制两次，在第二个维度上复制三次
y = tf.tile(x, [2, 3])

print(y)
```

这段代码将会输出：

```
[[1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]
 [1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]]
```

可以看到，`tf.tile` 在指定维度上复制了输入张量。

In [7]:
import tensorflow as tf

# 定义一个形状为 (2, 3) 的张量
x = tf.constant([[1, 2, 3], [4, 5, 6]])

# 在第一个维度上复制两次，在第二个维度上复制三次
y = tf.tile(x, [2, 3])

print(y)

tf.Tensor(
[[1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]
 [1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]], shape=(4, 9), dtype=int32)


# 应用

In [8]:
rayleigh = RayleighBlockFading(num_rx = 1,
                               num_rx_ant = 32,
                               num_tx = 4,
                               num_tx_ant = 2)
h, delays = rayleigh(batch_size=10, num_time_steps=100)
print(tf.shape(h), tf.shape(delays))

tf.Tensor([ 10   1  32   4   2   1 100], shape=(7,), dtype=int32) tf.Tensor([10  1  4  1], shape=(4,), dtype=int32)


# pytorch 重写

In [24]:
import torch
from my_code.mysionna.channel.torch_version.channel_model import ChannelModel

class RayleighBlockFading(ChannelModel):
    def __init__(self, 
                 num_rx, 
                 num_rx_ant, 
                 num_tx, 
                 num_tx_ant,
                 dtype= torch.complex64):
        
        assert dtype.is_complex,"'dtype' must be complex type"
        self._dtype = dtype

        self.num_tx = num_tx
        self.num_tx_ant = num_tx_ant
        self.num_rx = num_rx
        self.num_rx_ant = num_rx_ant

    def __call__(self, batch_size, num_time_steps, sampling_frequency=None):
        # Delays
        if self._dtype ==torch.complex32:
            real_dtype = torch.float16
        elif self._dtype == torch.complex64:
            real_dtype = torch.float32
        elif self._dtype == torch.complex128:
            real_dtype = torch.float64
        else: 
            raise TypeError("Not found complex dtype in [complex32,complex64,complex128]")
        
        delays = torch.zeros([batch_size, 
                              self.num_rx, 
                              self.num_tx, 
                              1],
                              dtype=real_dtype)

        # Fading coefficients
        # std = torch.tensor(torch.sqrt(torch.tensor(0.5)),dtype=real_dtype)
        std = torch.sqrt(torch.tensor(0.5)).type(dtype=real_dtype)
        h_real = torch.normal(mean=0.0,
                              std=std,
                              size=(batch_size, 
                                    self.num_rx, 
                                    self.num_rx_ant, 
                                    self.num_tx, 
                                    self.num_tx_ant, 
                                    1, 
                                    1),
                            dtype=real_dtype) 
        h_img = torch.normal(mean=0.0,
                             std=std,
                             size=(batch_size,
                                   self.num_rx,
                                   self.num_rx_ant,
                                   self.num_tx,
                                   self.num_tx_ant,
                                   1,
                                   1),
                            dtype=real_dtype)
        h = torch.complex(h_real, h_img)

        # Tile the response over the block
        h = h.repeat(1, 1, 1, 1, 1, 1, num_time_steps)
        return h, delays
    """     # Example usage
    num_rx = 2
    num_rx_ant = 2
    num_tx = 2
    num_tx_ant = 2
    dtype = torch.complex128

    rayleigh_block_fading = RayleighBlockFading(num_rx, num_rx_ant, num_tx, num_tx_ant, dtype)
    batch_size = 4
    num_time_steps = 10
    h, delays = rayleigh_block_fading(batch_size, num_time_steps)

    print(h.shape)  # Expected shape: [batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, 1, num_time_steps]
    print(delays.shape)  # Expected shape: [batch_size, num_rx, num_tx, 1] """

torch.Size([4, 2, 2, 2, 2, 1, 10])
torch.Size([4, 2, 2, 1])


In [26]:
rayleigh = RayleighBlockFading(num_rx = 1,
                               num_rx_ant = 32,
                               num_tx = 4,
                               num_tx_ant = 2)
h, delays = rayleigh(batch_size=10, num_time_steps=100)
print(h.shape)  # Expected shape: [batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, 1, num_time_steps]
print(delays.shape)  # Expected shape: [batch_size, num_rx, num_tx, 1]

torch.Size([10, 1, 32, 4, 2, 1, 100])
torch.Size([10, 1, 4, 1])


In [22]:
#
# SPDX-FileCopyrightText: Copyright (c) 2021-2023 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
"""Class for simulating Rayleigh block fading"""

import tensorflow as tf

from sionna.channel.channel_model import ChannelModel

class RayleighBlockFading(ChannelModel):
    # pylint: disable=line-too-long
    r"""RayleighBlockFading(num_rx, num_rx_ant, num_tx, num_tx_ant, dtype=tf.complex64)

    Generate channel impulse responses corresponding to a Rayleigh block
    fading channel model.

    The channel impulse responses generated are formed of a single path with
    zero delay and a normally distributed fading coefficient.
    All time steps of a batch example share the same channel coefficient
    (block fading).

    This class can be used in conjunction with the classes that simulate the
    channel response in time or frequency domain, i.e.,
    :class:`~sionna.channel.OFDMChannel`,
    :class:`~sionna.channel.TimeChannel`,
    :class:`~sionna.channel.GenerateOFDMChannel`,
    :class:`~sionna.channel.ApplyOFDMChannel`,
    :class:`~sionna.channel.GenerateTimeChannel`,
    :class:`~sionna.channel.ApplyTimeChannel`.

    Parameters
    ----------

    num_rx : int
        Number of receivers (:math:`N_R`)

    num_rx_ant : int
        Number of antennas per receiver (:math:`N_{RA}`)

    num_tx : int
        Number of transmitters (:math:`N_T`)

    num_tx_ant : int
        Number of antennas per transmitter (:math:`N_{TA}`)

    dtype : tf.DType
        Complex datatype to use for internal processing and output.
        Defaults to `tf.complex64`.

    Input
    -----
    batch_size : int
        Batch size

    num_time_steps : int
        Number of time steps

    Output
    -------
    a : [batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths = 1, num_time_steps], tf.complex
        Path coefficients

    tau : [batch size, num_rx, num_tx, num_paths = 1], tf.float
        Path delays [s]
    """

    def __init__(   self,
                    num_rx,
                    num_rx_ant,
                    num_tx,
                    num_tx_ant,
                    dtype=tf.complex64):

        assert dtype.is_complex, "'dtype' must be complex type"
        self._dtype = dtype

        # We don't set these attributes as private so that the user can update
        # them
        self.num_tx = num_tx
        self.num_tx_ant = num_tx_ant
        self.num_rx = num_rx
        self.num_rx_ant = num_rx_ant

    def __call__(self,  batch_size, num_time_steps, sampling_frequency=None):

        # Delays
        # Single path with zero delay
        delays = tf.zeros([ batch_size,
                            self.num_rx,
                            self.num_tx,
                            1], # Single path
                            dtype=self._dtype.real_dtype)

        # Fading coefficients
        std = tf.cast(tf.sqrt(0.5), dtype=self._dtype.real_dtype)
        h_real = tf.random.normal(shape=[   batch_size,
                                            self.num_rx,
                                            self.num_rx_ant,
                                            self.num_tx,
                                            self.num_tx_ant,
                                            1, # One path
                                            1], # Same response over the block
                                            stddev=std,
                                            dtype = self._dtype.real_dtype)
        h_img = tf.random.normal(shape=[    batch_size,
                                            self.num_rx,
                                            self.num_rx_ant,
                                            self.num_tx,
                                            self.num_tx_ant,
                                            1, # One cluster
                                            1], # Same response over the block
                                            stddev=std,
                                            dtype = self._dtype.real_dtype)
        h = tf.complex(h_real, h_img)
        # Tile the response over the block
        h = tf.tile(h, [1, 1, 1, 1, 1, 1, num_time_steps])
        return h, delays
    
    """ # Example usage
    num_rx = 2
    num_rx_ant = 2
    num_tx = 2
    num_tx_ant = 2
    dtype = tf.complex128

    rayleigh_block_fading = RayleighBlockFading(num_rx, num_rx_ant, num_tx, num_tx_ant, dtype)
    batch_size = 4
    num_time_steps = 10
    h, delays = rayleigh_block_fading(batch_size, num_time_steps)

    print(h.shape)  # Expected shape: [batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, 1, num_time_steps]
    print(delays.shape)  # Expected shape: [batch_size, num_rx, num_tx, 1] """


' # Example usage\nnum_rx = 2\nnum_rx_ant = 2\nnum_tx = 2\nnum_tx_ant = 2\ndtype = tf.complex128\n\nrayleigh_block_fading = RayleighBlockFading(num_rx, num_rx_ant, num_tx, num_tx_ant, dtype)\nbatch_size = 4\nnum_time_steps = 10\nh, delays = rayleigh_block_fading(batch_size, num_time_steps)\n\nprint(h.shape)  # Expected shape: [batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, 1, num_time_steps]\nprint(delays.shape)  # Expected shape: [batch_size, num_rx, num_tx, 1] '

In [23]:
rayleigh = RayleighBlockFading(num_rx = 1,
                               num_rx_ant = 32,
                               num_tx = 4,
                               num_tx_ant = 2)
h, delays = rayleigh(batch_size=10, num_time_steps=100)
print(tf.shape(h), tf.shape(delays))

tf.Tensor([ 10   1  32   4   2   1 100], shape=(7,), dtype=int32) tf.Tensor([10  1  4  1], shape=(4,), dtype=int32)
